In [58]:
import keras
import numpy as np

from keras.layers import Conv2D, Conv2DTranspose, MaxPool2D, Dense, Flatten, Dropout, BatchNormalization, Activation
from keras.models import Sequential

from keras.metrics import *
from keras.optimizers import Adam, SGD
from keras.callbacks import ModelCheckpoint
from keras.callbacks import LearningRateScheduler, EarlyStopping

# Image size: 256, 256, 1

# 1, 2, 8, 16, 32, 64, 128, 256, 512
import tensorflow as tf
from sklearn.metrics import roc_auc_score

import pandas as pd
from src.models.util.callbacks import Metrics
from src.helper import get_config

In [62]:
import yaml
with open("config.yaml") as f:
    config = yaml.load(f)
    
config

{'Section1': {'name': 'section 1', 'age': 20},
 'Section2': {'name': 'section 2', 'age': 3.4}}

In [2]:


def auroc(y_true, y_pred):
    
    try:
        tf.py_func(roc_auc_score, (y_true, y_pred), tf.double)
        return tf.py_func(roc_auc_score, (y_true, y_pred), tf.double)
    except:
        return

In [4]:
def FCCN():

    model = Sequential()

    model.add(Conv2D(input_shape=(64,64,3), strides=1, filters= 33, kernel_size=3, padding="same"))
    model.add(Conv2D(3, 1, padding="same"))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(2))



    model.add(Conv2D(strides=1, filters= 64, kernel_size=3, padding="same"))
    model.add(Conv2D(3, 1, padding= "same"))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(2))

    model.add(Conv2D(strides=1, filters= 128, kernel_size=3, padding= "same"))
    model.add(Conv2D(3, 1, padding="same"))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(2))
    
    model.add(Flatten())


    model.add(Dense(256, activation='sigmoid'))
    model.add(Dropout(0.5))
    #model.add(Dense(64, activation='sigmoid'))
    model.add(Dense(1, activation='sigmoid'))

    return model

In [5]:
model = FCCN()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 64, 64, 33)        924       
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 64, 64, 3)         102       
_________________________________________________________________
activation_4 (Activation)    (None, 64, 64, 3)         0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 64, 64, 3)         12        
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 32, 32, 64)        1792      
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 32, 32, 3)         195       
__________

In [6]:
X = np.load("../data/processed/X_2c.npy")
y = np.load("../data/processed/y_2c.npy")

In [15]:
np.mean(X)

-5.250273439228904e-17

In [16]:
X.shape

(296, 64, 64, 3)

In [7]:
X_train = X[:200,: ,:,:]
X_val = X[200:, :,::]


y_train = y[:200]
y_val = y[200:]

In [43]:
model_checkpoint = ModelCheckpoint('../data/weights.h5', monitor='val_loss', save_best_only=True)


metric = Metrics()
c_backs = [model_checkpoint]
c_backs.append( EarlyStopping(monitor='val_loss', min_delta=0.001, patience=5) )
c_backs.append( metric )
model.compile( optimizer=Adam(lr=0.001), loss='binary_crossentropy')

model.fit(X_train, y_train,
          batch_size=20,
          epochs=50,
          validation_data=(X_val, y_val),
          shuffle=True,
         callbacks=c_backs)

Train on 200 samples, validate on 96 samples
Epoch 1/50
200/200 [==============================] - 2s 12ms/step - loss: 0.1108 - val_loss: 0.9549
- val_f1: 0.1538 - val_precision: 0.1875 - val_recall 0.1304 - val_auroc 0.4762 
Epoch 2/50
200/200 [==============================] - 1s 7ms/step - loss: 0.0990 - val_loss: 1.1078
- val_f1: 0.0588 - val_precision: 0.0909 - val_recall 0.0435 - val_auroc 0.4532 
Epoch 3/50
200/200 [==============================] - 1s 7ms/step - loss: 0.0976 - val_loss: 1.0888
- val_f1: 0.2128 - val_precision: 0.2083 - val_recall 0.2174 - val_auroc 0.4786 
Epoch 4/50
200/200 [==============================] - 1s 7ms/step - loss: 0.0641 - val_loss: 1.1333
- val_f1: 0.0667 - val_precision: 0.1429 - val_recall 0.0435 - val_auroc 0.4806 
Epoch 5/50
200/200 [==============================] - 1s 7ms/step - loss: 0.0684 - val_loss: 1.2555
- val_f1: 0.0606 - val_precision: 0.1 - val_recall 0.0435 - val_auroc 0.4601 
Epoch 6/50
200/200 [==============================] 

<bound method Model.compile of <keras.engine.sequential.Sequential object at 0x7f029872c588>>

In [44]:
metric.val

[0.4762, 0.4532, 0.4786, 0.4806, 0.4601, 0.4887]

In [106]:
metric.__class__.__name__

'Metrics'

In [100]:
int(now.timestamp())

1539082503

In [71]:
hist = model.history.history
hist["epoch"] = model.history.epoch
hist["val_auroc"] = metric.val_auroc
hist["val_precision"] = metric.val_precisions
hist["val_recall"] = metric.val_recalls

In [54]:
hist["epoch"] = model.history.epoch
hist["val_auroc"] = metric.val_auroc
hist["val_precision"] = metric.val_precisions
hist["val_recall"] = metric.val_recalls

In [103]:
import datetime

now = datetime.datetime.now()

log = dict()
log["datetime"] = {'date': now.strftime("%Y-%m-%d %H:%M"), 'unix' : now.timestamp()}
log["parameters"] = config
log["history"] = hist

In [104]:
print(hist.to_json())

AttributeError: 'dict' object has no attribute 'to_json'

In [105]:
log

{'datetime': {'date': '2018-10-09 11:59', 'unix': 1539082750.907701},
 'parameters': {'Section1': {'name': 'section 1', 'age': 20},
  'Section2': {'name': 'section 2', 'age': 3.4}},
 'history': {'val_loss': [0.9549218155443668,
   1.107801587631305,
   1.0887549668550491,
   1.1332847625017166,
   1.255474613358577,
   1.2716096825897694],
  'loss': [0.11080916561186313,
   0.09900723025202751,
   0.09763280674815178,
   0.06411000154912472,
   0.06840433236211538,
   0.05415494218468666],
  'val_auroc': [0.4762, 0.4532, 0.4786, 0.4806, 0.4601, 0.4887],
  'val_precision': [0.1875, 0.0909, 0.2083, 0.1429, 0.1, 0.2],
  'val_recall': [0.1304, 0.0435, 0.2174, 0.0435, 0.0435, 0.087],
  'epoch': [0, 1, 2, 3, 4, 5]}}